# GRPC Inference

## Setup

Verify that following variable settings match your deployed model's resource name and grpc URL. The following code assumes that the kube service is in the same namespace, but you could refer to it in full with the namespace, for example: `http://modelmesh-serving.project-name.svc.cluster.local:8008/v2/models/fraud/infer`

In [ ]:
!pip install grpcio grpcio-tools

In [ ]:
grpc_host = 'modelmesh-serving.lakefs'
grpc_port = 8033
model_name = 'fraud'

## Inspect the gRPC Endpoint

Check the gRPC endpoint's model metadata:

In [ ]:
import sys
sys.path.append('./utils')

# grpc_predict_v2_pb2 and grpc_predict_v2_pb2_grpc were created from grpc_predict_v2.proto using protoc
import grpc
import utils.grpc_predict_v2_pb2 as grpc_predict_v2_pb2
import utils.grpc_predict_v2_pb2_grpc as grpc_predict_v2_pb2_grpc


channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
response

### Request Function

Build and submit the gRPC request. 

Note: You submit the data in the same format that you used for an ONNX inference.


In [ ]:
import numpy as np

def grpc_request(data):
    # request content building
    inputs = []
    inputs.append(grpc_predict_v2_pb2.ModelInferRequest().InferInputTensor())
    inputs[0].name = "dense_input"
    inputs[0].datatype = "FP32"
    inputs[0].shape.extend([1, 5])
    inputs[0].contents.fp32_contents.extend(data)

    # request building
    request = grpc_predict_v2_pb2.ModelInferRequest()
    request.model_name = model_name
    request.inputs.extend(inputs)

    response = stub.ModelInfer(request)
    result_arr = np.frombuffer(response.raw_output_contents[0], dtype=np.float32)
    return result_arr

### Run the Request

In [ ]:
# Define lakeFS Repository
import os
import lakefs

repo_name = os.environ.get('LAKEFS_REPO_NAME')

mainBranch = "main"
trainingBranch = "train01"

repo = lakefs.Repository(repo_name)
branchTraining = repo.branch(trainingBranch)
print(repo)

In [ ]:
#Load the scaler from the training branch in lakeFS
import pickle
obj = branchTraining.object(path='artifact/scaler.pkl')
with obj.reader("rb") as handle:
    scaler = pickle.load(handle)

In [ ]:
data = [0.3111400080477545, 1.9459399775518593, 1.0, 0.0, 0.0]
prediction = grpc_request(scaler.transform([data]).tolist()[0])
prediction

In [ ]:
threshhold = 0.95

if (prediction[0] > threshhold):
    print('fraud')
else:
    print('not fraud')

## Example 1: user buys a coffee

In this example, the user is buying a coffee. The parameters given to the model are:
* same location as the last transaction (distance=0)
* same median price as the last transaction (ratio_to_median=1)
* using a pin number (pin=1)
* using the credit card chip (chip=1)
* not an online transaction (online=0)

In [ ]:
data = [0.0, 1.0, 1.0, 1.0, 0.0]
prediction = grpc_request(scaler.transform([data]).tolist()[0])
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')

## Example 2: fraudulent transaction

In this example, someone stole the user's credit card and is buying something online. The parameters given to the model are:
* very far away from the last transaction (distance=100)
* median price similar to the last transaction (ratio_to_median=1.2)
* not using a pin number (pin=0)
* not using the credit card chip (chip=0)
* is an online transaction (online=1)

In [ ]:
data = [100, 1.2, 0.0, 0.0, 1.0]
prediction = grpc_request(scaler.transform([data]).tolist()[0])
threshhold = 0.95

if (prediction[0] > threshhold):
    print('The model predicts that this is fraud')
else:
    print('The model predicts that this is not fraud')